In [ ]:
!apt-get update                                                                          # Update Package 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null                                  # install Java Development Kit (important!!! cant run Spark with out this)
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz # install Spark 3.1.2
!tar xzvf spark-3.1.2-bin-hadoop2.7.tgz                                                  # Unzip  Spark 3.1.2
!pip install -q findspark==1.3.0                                                         # install Package Python for connect with Spark 

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import pandas as pd

In [5]:
!pip install pyspark==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 70 kB/s 
     |████████████████████████████████| 198 kB 43.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=c65aa52298584e8c6b00985b4be421ac2073243ade40641bcd78ce6bc9291ef0
  Stored in directory: /root/.cache/pip/wheels/df/88/9e/58ef1f74892fef590330ca0830b5b6d995ba29b44f977b3926
Successfully built pyspark


In [6]:
# Create Spark Session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
#  Python Version
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=15, releaselevel='final', serial=0)

In [8]:
# Download Data File !wget

!wget https://maven-datasets.s3.amazonaws.com/Wine+Tasting/winemag-data-130k-v2.csv.zip

--2022-12-02 14:13:23--  https://maven-datasets.s3.amazonaws.com/Wine+Tasting/winemag-data-130k-v2.csv.zip
Resolving maven-datasets.s3.amazonaws.com (maven-datasets.s3.amazonaws.com)... 52.217.193.217, 52.217.172.233, 52.216.221.225, ...
Connecting to maven-datasets.s3.amazonaws.com (maven-datasets.s3.amazonaws.com)|52.217.193.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16934504 (16M) [application/zip]
Saving to: ‘winemag-data-130k-v2.csv.zip’

winemag-data-130k-v 100%[===================>]  16.15M  78.2MB/s    in 0.2s    

2022-12-02 14:13:23 (78.2 MB/s) - ‘winemag-data-130k-v2.csv.zip’ saved [16934504/16934504]



In [9]:
!unzip winemag-data-130k-v2.csv.zip

Archive:  winemag-data-130k-v2.csv.zip
  inflating: winemag-data-130k-v2.csv  
  inflating: __MACOSX/._winemag-data-130k-v2.csv  


In [75]:
dt = spark.read.csv('/content/winemag-data-130k-v2.csv', header = True, inferSchema = True, )

In [11]:
# check column
dt

DataFrame[id: string, country: string, description: string, designation: string, points: string, price: string, province: string, region_1: string, region_2: string, taster_name: string, taster_twitter_handle: string, title: string, variety: string, winery: string]

In [12]:
dt.dtypes

[('id', 'string'),
 ('country', 'string'),
 ('description', 'string'),
 ('designation', 'string'),
 ('points', 'string'),
 ('price', 'string'),
 ('province', 'string'),
 ('region_1', 'string'),
 ('region_2', 'string'),
 ('taster_name', 'string'),
 ('taster_twitter_handle', 'string'),
 ('title', 'string'),
 ('variety', 'string'),
 ('winery', 'string')]

In [13]:
dt.printSchema()

root
 |-- id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [14]:
dt.show()

+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
| id|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1| Portugal|This is ripe and ...|            Avidagos|    87|   15|            Douro|

In [15]:
print((dt.count(), len(dt.columns)))

(129975, 14)


In [51]:
dt.dropDuplicates().count()


129975

In [37]:
dt.summary().show()

+-------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+
|summary|               id|             country|         description|         designation|              points|               price|            province|            region_1|            region_2|         taster_name|taster_twitter_handle|               title|             variety|              winery|
+-------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+
|  count|           129975|              129912|              129974|               92513|    

In [38]:
dt.summary("count").show()

+-------+------+-------+-----------+-----------+------+------+--------+--------+--------+-----------+---------------------+------+-------+------+
|summary|    id|country|description|designation|points| price|province|region_1|region_2|taster_name|taster_twitter_handle| title|variety|winery|
+-------+------+-------+-----------+-----------+------+------+--------+--------+--------+-----------+---------------------+------+-------+------+
|  count|129975| 129912|     129974|      92513|129966|120976|  129909|  108734|   50521|     103726|                98762|129965| 129963|129963|
+-------+------+-------+-----------+-----------+------+------+--------+--------+--------+-----------+---------------------+------+-------+------+



In [47]:
dt = dt.drop('region_1','region_2','region_')
dt.show()

+---+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
| id|  country|         description|         designation|points|price|         province|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1| Portugal|This is ripe and ...|            Avidagos|    87|   15|            Douro|        Roger Voss|           @vossroger|Quinta dos Avidag...|    Portuguese Red|Quinta dos Avidagos|
|  2|       US|Tart and snappy, ...|              

In [76]:
dt.where( dt.points.isNull()).show()

+--------------------+--------------------+--------------------+--------------------+------+-----+-------------+------------------+--------------------+-----------+---------------------+------+--------------------+------------+
|                  id|             country|         description|         designation|points|price|     province|          region_1|            region_2|taster_name|taster_twitter_handle| title|             variety|      winery|
+--------------------+--------------------+--------------------+--------------------+------+-----+-------------+------------------+--------------------+-----------+---------------------+------+--------------------+------------+
|               18881|        South Africa|Notes of cocoa an...|Grand Classique C...|  null| null|         null|              null|                null|       null|                 null|  null|                null|        null|
|Merlot-Cabernet F...|                  90|                  20|               Paarl|  n

In [78]:
dt.groupBy('taster_name').count().orderBy('count', ascending = False).show(38)

+--------------------+-----+
|         taster_name|count|
+--------------------+-----+
|                null|26249|
|          Roger Voss|25513|
|   Michael Schachner|15133|
|       Kerin O’Keefe|10776|
|      Virginie Boone| 9537|
|        Paul Gregutt| 9529|
|       Matt Kettmann| 6332|
|      Joe Czerwinski| 5139|
|    Sean P. Sullivan| 4966|
|  Anna Lee C. Iijima| 4415|
|          Jim Gordon| 4177|
|    Anne Krebiehl MW| 3683|
|       Lauren Buzzeo| 1832|
|     Susan Kostrzewa| 1085|
|       Mike DeSimone|  514|
|        Jeff Jenssen|  491|
|  Alexander Peartree|  415|
|        Carrie Dykes|  139|
|         Fiona Adams|   27|
|   Christina Pickard|    6|
|                  89|    2|
|   Chambolle-Musigny|    1|
|                Etna|    1|
|                  84|    1|
|                  95|    1|
| there's remarkab...|    1|
|         Napa Valley|    1|
| almost severe el...|    1|
| but you can cert...|    1|
| mocha and cinnam...|    1|
|                  86|    1|
|       Centra

In [70]:
dt_c = dt.groupBy('taster_name').count().orderBy('count', ascending = False)
dt_c = dt_c.where('count == 1')
dt_c.show()

+--------------------+-----+
|         taster_name|count|
+--------------------+-----+
|   Chambolle-Musigny|    1|
|                Etna|    1|
| there's remarkab...|    1|
|                  95|    1|
| mocha and cinnam...|    1|
|                  25|    1|
|                  86|    1|
|                  84|    1|
|         Napa Valley|    1|
| almost severe el...|    1|
| but you can cert...|    1|
|          California|    1|
|       Central Coast|    1|
|        Finger Lakes|    1|
|Edition Chremisa ...|    1|
+--------------------+-----+



In [61]:
from pyspark.sql import functions as F, Window as W


In [80]:
window = W.partitionBy("taster_name")

dt_name = dt.withColumn("count", F.count("taster_name").over(window))\
              .filter(F.col("count") > 2).drop("count")
dt_name.groupBy('taster_name').count().orderBy('count', ascending = False).show(38)

+------------------+-----+
|       taster_name|count|
+------------------+-----+
|        Roger Voss|25513|
| Michael Schachner|15133|
|     Kerin O’Keefe|10776|
|    Virginie Boone| 9537|
|      Paul Gregutt| 9529|
|     Matt Kettmann| 6332|
|    Joe Czerwinski| 5139|
|  Sean P. Sullivan| 4966|
|Anna Lee C. Iijima| 4415|
|        Jim Gordon| 4177|
|  Anne Krebiehl MW| 3683|
|     Lauren Buzzeo| 1832|
|   Susan Kostrzewa| 1085|
|     Mike DeSimone|  514|
|      Jeff Jenssen|  491|
|Alexander Peartree|  415|
|      Carrie Dykes|  139|
|       Fiona Adams|   27|
| Christina Pickard|    6|
+------------------+-----+



In [86]:
dt = dt_name
dt.groupBy('taster_name').count().orderBy('count', ascending = False).show()

+------------------+-----+
|       taster_name|count|
+------------------+-----+
|        Roger Voss|25513|
| Michael Schachner|15133|
|     Kerin O’Keefe|10776|
|    Virginie Boone| 9537|
|      Paul Gregutt| 9529|
|     Matt Kettmann| 6332|
|    Joe Czerwinski| 5139|
|  Sean P. Sullivan| 4966|
|Anna Lee C. Iijima| 4415|
|        Jim Gordon| 4177|
|  Anne Krebiehl MW| 3683|
|     Lauren Buzzeo| 1832|
|   Susan Kostrzewa| 1085|
|     Mike DeSimone|  514|
|      Jeff Jenssen|  491|
|Alexander Peartree|  415|
|      Carrie Dykes|  139|
|       Fiona Adams|   27|
| Christina Pickard|    6|
+------------------+-----+



In [19]:
dt.select("country").distinct().count()

48

In [60]:
dt.groupBy('country').count().orderBy('count', ascending = False).show(48, False)

+----------------------------------------------------------------------------------+-----+
|country                                                                           |count|
+----------------------------------------------------------------------------------+-----+
|US                                                                                |54504|
|France                                                                            |22093|
|Italy                                                                             |19540|
|Spain                                                                             |6645 |
|Portugal                                                                          |5691 |
|Chile                                                                             |4472 |
|Argentina                                                                         |3800 |
|Austria                                                                           |3345 |

In [83]:
window_c = W.partitionBy("country")

dt_country = dt.withColumn("count", F.count("country").over(window_c))\
              .filter(F.col("count") > 1).drop("count")
dt_country.groupBy('country').count().orderBy('count', ascending = False).show(48)

+--------------------+-----+
|             country|count|
+--------------------+-----+
|                  US|54504|
|              France|22093|
|               Italy|19540|
|               Spain| 6645|
|            Portugal| 5691|
|               Chile| 4472|
|           Argentina| 3800|
|             Austria| 3345|
|           Australia| 2329|
|             Germany| 2165|
|         New Zealand| 1419|
|        South Africa| 1401|
|              Israel|  505|
|              Greece|  466|
|              Canada|  257|
|             Hungary|  146|
|            Bulgaria|  141|
|             Romania|  120|
|             Uruguay|  109|
|              Turkey|   90|
|            Slovenia|   87|
|             Georgia|   86|
|             England|   74|
|             Croatia|   73|
|              Mexico|   70|
|             Moldova|   59|
|              Brazil|   52|
|             Lebanon|   35|
|             Morocco|   28|
|                Peru|   16|
|             Ukraine|   14|
|             

In [89]:
dt = dt_country 
dt.groupBy('country').count().orderBy('count', ascending = False).show(48)

+--------------------+-----+
|             country|count|
+--------------------+-----+
|                  US|54504|
|              France|22093|
|               Italy|19540|
|               Spain| 6645|
|            Portugal| 5691|
|               Chile| 4472|
|           Argentina| 3800|
|             Austria| 3345|
|           Australia| 2329|
|             Germany| 2165|
|         New Zealand| 1419|
|        South Africa| 1401|
|              Israel|  505|
|              Greece|  466|
|              Canada|  257|
|             Hungary|  146|
|            Bulgaria|  141|
|             Romania|  120|
|             Uruguay|  109|
|              Turkey|   90|
|            Slovenia|   87|
|             Georgia|   86|
|             England|   74|
|             Croatia|   73|
|              Mexico|   70|
|             Moldova|   59|
|              Brazil|   52|
|             Lebanon|   35|
|             Morocco|   28|
|                Peru|   16|
|             Ukraine|   14|
|             

In [21]:
dt.select("country").distinct().sort("country").show( 48, False )

+----------------------------------------------------------------------------------+
|country                                                                           |
+----------------------------------------------------------------------------------+
|null                                                                              |
| fine and extremely polished; hold for 10 years."                                 |
| marked by tart citrus flavors that make it versatile with a wide range of foods."|
|90                                                                                |
|Argentina                                                                         |
|Armenia                                                                           |
|Australia                                                                         |
|Austria                                                                           |
|Bordeaux-style Red Blend                                        

In [22]:
dt_clean = dt.filter("country IS NULL").show()

+-----+-------+--------------------+--------------------+------+-----+--------+--------+--------+-----------------+---------------------+--------------------+---------------+--------------------+
|   id|country|         description|         designation|points|price|province|region_1|region_2|      taster_name|taster_twitter_handle|               title|        variety|              winery|
+-----+-------+--------------------+--------------------+------+-----+--------+--------+--------+-----------------+---------------------+--------------------+---------------+--------------------+
|  913|   null|Amber in color, t...|      Asureti Valley|    87|   30|    null|    null|    null|    Mike DeSimone|       @worldwineguys|Gotsa Family Wine...|        Chinuri|  Gotsa Family Wines|
| 3131|   null|Soft, fruity and ...|            Partager|    83| null|    null|    null|    null|       Roger Voss|           @vossroger|Barton & Guestier...|      Red Blend|   Barton & Guestier|
| 4243|   null|Viole

In [23]:
dt.where('''country in ('90',
                        ' fine and extremely polished; hold for 10 years."',
                        ' marked by tart citrus flavors that make it versatile with a wide range of foods."',
                        'Bordeaux-style Red Blend')''').show()

+--------------------+--------------------+-----------+-----------+------+----------+-------------+----------------+--------------------+-----------+---------------------+------------+-----------+------+
|                  id|             country|description|designation|points|     price|     province|        region_1|            region_2|taster_name|taster_twitter_handle|       title|    variety|winery|
+--------------------+--------------------+-----------+-----------+------+----------+-------------+----------------+--------------------+-----------+---------------------+------------+-----------+------+
|Merlot-Cabernet F...|                  90|         20|      Paarl|  null|      null|Lauren Buzzeo|       @laurbuzz|Glen Carlou 2009 ...|       null|                 null|        null|       null|  null|
|Merlot-Cabernet F...|Bordeaux-style Re...|Glen Carlou|       null|  null|      null|         null|            null|                null|       null|                 null|        null|